In [18]:
import sqlite3
from sqlite3 import Error

## Função de alteração (insert, update, delete...)
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Query {query.split('(')[0].strip()} executada.")
        if cursor.rowcount != -1: print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}'")


# Função de leitura de dados (select)
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

In [19]:
## Criando conexão
connection = sqlite3.connect("mapeamento_cultural.sqlite")

query = """
CREATE TABLE IF NOT EXISTS endereco (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    rua VARCHAR(100),
    numero VARCHAR(10),
    CEP VARCHAR(10),
    regiao VARCHAR(50)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE IF NOT EXISTS pontos_historicos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    descricao TEXT,
    endereco_id INT,
    FOREIGN KEY (endereco_id) REFERENCES endereco(id)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE IF NOT EXISTS grupo_cultural (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome VARCHAR(100),
    descricao TEXT,
    fotos TEXT,
    contato VARCHAR(100)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE IF NOT EXISTS evento (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    data DATE,
    horario TIME,
    local VARCHAR(100),
    status VARCHAR(50),
    url_ingresso VARCHAR(255)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE IF NOT EXISTS localiza (
    grupo_id INTEGER,
    ponto_id INTEGER,
    PRIMARY KEY (grupo_id, ponto_id),
    FOREIGN KEY (grupo_id) REFERENCES grupo_cultural(id),
    FOREIGN KEY (ponto_id) REFERENCES pontos_historicos(id)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE IF NOT EXISTS apresenta (
    grupo_id INTEGER,
    evento_id INTEGER,
    grupo_responsavel VARCHAR(100),
    PRIMARY KEY (grupo_id, evento_id),
    FOREIGN KEY (grupo_id) REFERENCES grupo_cultural(id),
    FOREIGN KEY (evento_id) REFERENCES evento(id)
);
"""
execute_query(connection, query)

Query CREATE TABLE IF NOT EXISTS endereco executada.
Query CREATE TABLE IF NOT EXISTS pontos_historicos executada.
Query CREATE TABLE IF NOT EXISTS grupo_cultural executada.
Query CREATE TABLE IF NOT EXISTS evento executada.
Query CREATE TABLE IF NOT EXISTS localiza executada.
Query CREATE TABLE IF NOT EXISTS apresenta executada.


In [20]:
query = """
SELECT
    ph.id AS ponto_id,
    ph.descricao AS descricao_ponto,
    COUNT(l.grupo_id) AS total_grupos
FROM
    pontos_historicos ph
LEFT JOIN
    localiza l ON ph.id = l.ponto_id
GROUP BY
    ph.id, ph.descricao
ORDER BY
    total_grupos DESC;
"""

In [21]:
query = """
SELECT
    gc.id AS grupo_id,
    gc.nome AS nome_grupo,
    COUNT(a.evento_id) AS total_eventos
FROM
    grupo_cultural gc
LEFT JOIN
    apresenta a ON gc.id = a.grupo_id
GROUP BY
    gc.id, gc.nome
ORDER BY
    total_eventos DESC;
"""

In [22]:
create_view_query = """
CREATE VIEW IF NOT EXISTS view_eventos_com_grupos_e_locais AS
SELECT
    e.id AS evento_id,
    e.data,
    e.horario,
    e.local,
    gc.nome AS nome_grupo,
    ph.descricao AS ponto_historico,
    ed.rua,
    ed.numero,
    ed.regiao
FROM
    evento e
JOIN
    apresenta a ON e.id = a.evento_id
JOIN
    grupo_cultural gc ON a.grupo_id = gc.id
JOIN
    localiza l ON gc.id = l.grupo_id
JOIN
    pontos_historicos ph ON l.ponto_id = ph.id
JOIN
    endereco ed ON ph.endereco_id = ed.id
WHERE
    e.status = 'ativo';
"""

execute_query(connection, create_view_query)

Query CREATE VIEW IF NOT EXISTS view_eventos_com_grupos_e_locais AS
SELECT
    e.id AS evento_id,
    e.data,
    e.horario,
    e.local,
    gc.nome AS nome_grupo,
    ph.descricao AS ponto_historico,
    ed.rua,
    ed.numero,
    ed.regiao
FROM
    evento e
JOIN
    apresenta a ON e.id = a.evento_id
JOIN
    grupo_cultural gc ON a.grupo_id = gc.id
JOIN
    localiza l ON gc.id = l.grupo_id
JOIN
    pontos_historicos ph ON l.ponto_id = ph.id
JOIN
    endereco ed ON ph.endereco_id = ed.id
WHERE
    e.status = 'ativo'; executada.


In [23]:
def registrar_evento_com_grupo(connection, data, horario, local, status, url_ingresso, grupo_id, grupo_responsavel):
    cursor = connection.cursor()

    try:
        cursor.execute("SELECT id FROM grupo_cultural WHERE id = ?", (grupo_id,))
        if cursor.fetchone() is None:
            print("Erro: Grupo cultural não encontrado.")
            return

        cursor.execute("SELECT * FROM localiza WHERE grupo_id = ?", (grupo_id,))
        if cursor.fetchone() is None:
            print("Aviso: O grupo não está vinculado a nenhum ponto histórico.")

        cursor.execute("""
            INSERT INTO evento (data, horario, local, status, url_ingresso)
            VALUES (?, ?, ?, ?, ?)
        """, (data, horario, local, status, url_ingresso))
        evento_id = cursor.lastrowid

        cursor.execute("""
            INSERT INTO apresenta (grupo_id, evento_id, grupo_responsavel)
            VALUES (?, ?, ?)
        """, (grupo_id, evento_id, grupo_responsavel))

        connection.commit()
        print("Evento registrado e associado com sucesso.")

    except Error as e:
        print(f"Erro ao registrar evento: {e}")


In [24]:
def listar_eventos_por_grupo(connection, grupo_id):
    cursor = connection.cursor()
    try:
        cursor.execute("""
            SELECT
                e.id AS evento_id,
                e.data,
                e.horario,
                e.local,
                e.status,
                e.url_ingresso
            FROM evento e
            JOIN apresenta a ON e.id = a.evento_id
            WHERE a.grupo_id = ?
            ORDER BY e.data ASC
        """, (grupo_id,))
        resultados = cursor.fetchall()
        return resultados
    except Error as e:
        print(f"Erro ao buscar eventos do grupo: {e}")
        return []


In [25]:
query = """
CREATE TRIGGER IF NOT EXISTS inativar_evento_sem_grupo
AFTER DELETE ON apresenta
FOR EACH ROW
BEGIN
    UPDATE evento
    SET status = 'inativo'
    WHERE id = OLD.evento_id
    AND NOT EXISTS (
        SELECT 1 FROM apresenta WHERE evento_id = OLD.evento_id
    );
END;
"""
execute_query(connection, query)

Query CREATE TRIGGER IF NOT EXISTS inativar_evento_sem_grupo
AFTER DELETE ON apresenta
FOR EACH ROW
BEGIN
    UPDATE evento
    SET status = 'inativo'
    WHERE id = OLD.evento_id
    AND NOT EXISTS executada.
